# 詞神林夕養成計畫(keras)

![md_images](../Images/charrnn.png)

In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import *

from keras.optimizers import Adam
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import os


Using TensorFlow backend.


In [2]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF


def get_session(gpu_fraction=0.9):
    """
    This function is to allocate GPU memory a specific fraction
    Assume that you have 6GB of GPU memory and want to allocate ~2GB
    """
    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

KTF.set_session(get_session(0.9))  # using 40% of total GPU Memory

In [3]:
with io.open('lingxi.txt', encoding='utf-8-sig') as f:
    text = f.read().lower()

print('corpus length:', len(text))
texts=text.split('\n')
texts=[t for t in texts if len(t.replace(' ',''))>0]
print('corpus rows:', len(texts))
print(text[:5])
print(texts[:5])

corpus length: 52647
corpus rows: 4117
你說你 從
['你說你 從來未愛戀過', '但很珍惜 跟我在消磨', '我笑我 原來是我的錯', '裂開的心 還未算清楚', '如此天真 竟得我一個']


In [4]:
#把每個字去重複
chars = sorted(list(set(text)))
print('total chars:', len(chars))
print(chars[:10])


char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


total chars: 2114
['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '1']


In [5]:
maxlen = 40

print('Vectorization...')
x_features=[]
y_labels=[]
curr_chars=text[:-1]
next_chars=text[1:]
for i in range((len(curr_chars)-maxlen-1)//2):
    x_features.append(curr_chars[2*i:2*i+maxlen])
    y_labels.append(next_chars[2*i:2*i+maxlen])

x= np.zeros((len(x_features),maxlen,len(chars)), dtype=np.float32)
y = np.zeros((len(x_features),maxlen,len(chars)), dtype=np.float32)
    
for i in range(len(x_features)):
    for m in range(maxlen):
        x[i,m,char_indices[x_features[i][m]]]=1
        y[i,m,char_indices[y_labels[i][m]]]=1
  
print(x.shape)

Vectorization...
(26302, 40, 2114)


In [14]:
print('Build model...')


model = Sequential()
model.add(LSTM(512, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(LSTM(512, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))


optimizer =Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
if not os.path.exists('Models'):
    os.mkdir('Models')
    print("Directory Models Created ")
    
if os.path.exists('Models/lingxi_keras.h5'):
    model.load_weights('Models/lingxi_keras.h5')
    print('load model')
print (model.summary())

Build model...
load model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 40, 512)           5380096   
_________________________________________________________________
lstm_4 (LSTM)                (None, 40, 512)           2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 40, 2114)          1084482   
_________________________________________________________________
activation_2 (Activation)    (None, 40, 2114)          0         
Total params: 8,563,778
Trainable params: 8,563,778
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds+10e-14) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / (np.sum(exp_preds)+10e-14)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- 第Epoch: %d後自動寫詞' % epoch)
    model.save_weights('Models/lingxi_keras.h5',True)
    start_index = random.randint(0, len(texts))
    for diversity in [1.0]:
        print('----- diversity:', diversity)
        
        generated = ''
        sentence=''
        input_sentence=''
        sentence = texts[start_index]
        if len(sentence)>5:
            sentence=sentence[:5]
        generated += sentence
        print('----- 根據以下詞彙發想:「{0}」'.format(sentence ))
        sys.stdout.write(generated)
        is_finished=False
        row=0
        while not is_finished:
            x_pred = np.zeros((1, maxlen,len(chars)))
            if len(sentence)>maxlen-1:
                input_sentence=sentence[-1*(maxlen-1):]
            else:
                input_sentence=sentence
            for t, char in enumerate(input_sentence):
                x_pred[0, t,char_indices[char]] =1
            
            preds = model.predict(x_pred, verbose=0)[0]
          
            next_index =sample(preds[len(input_sentence)-1],1.0)
            next_char = indices_char[next_index]
            if next_char==' ' and len(generated)>5 and generated[-2:]=='  ':
                is_finished =True
                break
            generated += next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
            sentence = sentence+ next_char
 
            row+=1
            if row>100:
                is_finished =True
        print()

In [21]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=64,
          epochs=5,
          callbacks=[print_callback])



Epoch 1/5
26302/26302 [==============================] - 158s 6ms/step - loss: 1.2772 - acc: 0.6790

----- 第Epoch: 0後自動寫詞
----- diversity: 1.0
----- 根據以下詞彙發想:「跟你相對時」
跟你相對時分開
如果你愛我的清角 並有總是好 沒有心火
無謂一生 難得〕
不管你 我要眼睛
漆黑中的找到我
沒法想這個話 看不到
誰比難道會
但每一句的愛 你不會的眼前
對你結尾的傳笑
回頭動心的貼過
留下風光世
Epoch 2/5
26302/26302 [==============================] - 157s 6ms/step - loss: 1.1898 - acc: 0.6968

----- 第Epoch: 1後自動寫詞
----- diversity: 1.0
----- 根據以下詞彙發想:「一夜之間化」
一夜之間化做話 不要緊 你你去做證 你與你抱 我的你
怕不知 當你那份浪漫和寧靜
才可以端莊 也可以放浪
造型隨你幻想
你不愛你要 我要心得不好
我要給我最美滿太高興
現在就開始哭進在大國南
皇全你手 快樂地 隨
Epoch 3/5
26302/26302 [==============================] - 177s 7ms/step - loss: 1.1069 - acc: 0.7155

----- 第Epoch: 2後自動寫詞
----- diversity: 1.0
----- 根據以下詞彙發想:「每日要飲幾」
每日要飲幾罐汽水
你未與到掌聲之後我能選擇關呼一一方不覺
亦有任何事回怪兩點 從來給我自己
心在最後 為了會更深心
溫柔一然來相擁
從不可 貼過的人
牽著手 是吻別
但你前在為何沒有醒
我的身情 轉天怎不必懂
為
Epoch 4/5
26302/26302 [==============================] - 229s 9ms/step - loss: 1.0483 - acc: 0.7280

----- 第Epoch: 3後自動寫詞
----- diversity: 1.0
----- 根據以下詞彙發想:「然後一起分」
然後一起分離 看過了個世顧
不